In [201]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
import plotly.express as px
from python_module.pricing_model import SABRModel

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.float_format = '{:,.2f}'.format

In [202]:
# -------
# INPUTS
# -------

# Pricing parameters
F     = 5000
alpha = 0.25
beta  = 1.0
rho   = -0.
nu    = 1.5
r     = 0
atm_bd = 20
tail_bd = 20

# Scenario parameters
scenario_min = -0.3
scenario_max = 0.3
scenario_step = 0.05

option_strikes = [-0.05, 0.05]
flat_scenarios = [-0.3, 0.3]

In [203]:
# Scenario parameters
num = int((scenario_max-scenario_min)/0.01 + 1)
slide_to_compute = list(np.round(np.linspace(start=scenario_min, stop=scenario_max, num=num), 2))

# Option portfolio
market_data_list = list()

# Option ATM
K = F
T = atm_bd / 250
option_type = 'call' if K >= F else 'put'
market_pricing_results = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type, slide_list=slide_to_compute)
market_data_list.append({
    'symbol': f"{atm_bd}T_{K}K_{option_type}",
    'option_type': option_type, 
    'time_to_maturity': atm_bd, 
    'F':F, 'r': r, 'K': K, 'T': T, 'K/F': (K/F) - 1 ,**market_pricing_results})

for strike in option_strikes:
    K = F * (1+strike)
    T = tail_bd / 250
    option_type = 'call' if K >= F else 'put'
    market_pricing_results = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type, slide_list=slide_to_compute)
    market_data_list.append({
        'symbol': f"{tail_bd}T_{K}K_{option_type}",
        'option_type': option_type, 
        'time_to_maturity': tail_bd, 
        'F':F, 'r': r, 'K': K, 'T': T, 'K/F': (K/F) - 1 ,**market_pricing_results})

market_data_df = pd.DataFrame(market_data_list)
market_data_df = market_data_df.set_index('symbol')

In [204]:
# ATM Scenarios
atm_market_data = market_data_df.loc[market_data_df['F']==market_data_df['K']]
atm_qty = -50000 / atm_market_data['theta']
atm_scenario = atm_market_data.loc[:, ['theta'] + slide_to_compute]
atm_scenario = atm_scenario.drop(0, axis=1)
atm_scenario.rename(columns={'theta': 0}, inplace=True)
atm_scenario = atm_scenario.sort_index(axis=1)
atm_scenario = atm_scenario.multiply(atm_qty, axis=0)
atm_scenario = atm_scenario.iloc[0].transpose()
target_hedge_portfolio_scenario = atm_scenario*-1

In [205]:
# OTM Scenarios
otm_market_data = market_data_df.loc[market_data_df['F']!=market_data_df['K']]
otm_scenario = otm_market_data.loc[:, ['theta'] + slide_to_compute]
otm_scenario = otm_scenario.drop(0, axis=1)
otm_scenario.rename(columns={'theta': 0}, inplace=True)
otm_scenario = otm_scenario.sort_index(axis=1)
otm_scenario = otm_scenario.transpose()

In [206]:
X = otm_scenario.loc[flat_scenarios].values
b = target_hedge_portfolio_scenario.loc[flat_scenarios].values
w = np.linalg.solve(X, b)
weights = pd.Series(w, index=otm_market_data.index)
print(weights)

symbol
20T_4750.0K_put    -7,279.56
20T_5250.0K_call   -7,553.57
dtype: float64


In [207]:
total_scenario = atm_scenario + (otm_scenario * weights).sum(axis=1)

In [208]:
px.line(total_scenario)